In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset
from tqdm import tqdm
import random
import os

In [3]:
device = 'cuda'

In [4]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
seed_everything(42)

# Dataset

In [6]:
df_train = pd.read_csv('./data/train_result_mean_median_max_min_current_next_prev.csv')
df_test = pd.read_csv('./data/test_result_mean_median_max_min_current_next_prev.csv')

In [7]:
df_train.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_next_min,X4_next_min,X2_prev_min,X4_prev_min,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,780,905,867,931,954,935,40,1,2,7
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2110,2096,2224,2126,2026,2034,17,13,8,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1822,2153,2092,2047,1883,2053,1,1,2,4
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1416,1506,1564,1502,1474,1,1,1,26
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1263,1264,1239,1162,1202,1191,8,10,19,1


In [8]:
binarizer = MultiLabelBinarizer(classes=np.arange(0,101))

In [9]:
fig_1_train = binarizer.fit_transform(df_train.iloc[:, 6:14].as_matrix())
fig_2_train = binarizer.transform(df_train.iloc[:, 14:22].as_matrix())
fig_1_test = binarizer.transform(df_test.iloc[:, 6:14].as_matrix())
fig_2_test = binarizer.transform(df_test.iloc[:, 14:22].as_matrix())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [10]:
df_train['1/to_next_game_1'] = df_train['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_next_game_2'] = df_train['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_1'] = df_train['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_2'] = df_train['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

df_test['1/to_next_game_1'] = df_test['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_next_game_2'] = df_test['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_1'] = df_test['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_2'] = df_test['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

In [11]:
df_train

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2,1/to_next_game_1,1/to_next_game_2,1/to_prev_game_1,1/to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,954,935,40,1,2,7,0.025000,1.000000,0.500000,0.142857
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2026,2034,17,13,8,1,0.058824,0.076923,0.125000,1.000000
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1883,2053,1,1,2,4,1.000000,1.000000,0.500000,0.250000
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1474,1,1,1,26,1.000000,1.000000,1.000000,0.038462
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1202,1191,8,10,19,1,0.125000,0.100000,0.052632,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1211,1049,6,0,16,166,0.166667,0.000000,0.062500,0.006024
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,699,706,0,18,4,10,0.000000,0.055556,0.250000,0.100000
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1063,1036,5,1,257,0,0.200000,1.000000,0.003891,0.000000
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1752,1719,0,1,4,1,0.000000,1.000000,0.250000,1.000000


In [12]:
rating_1_train = df_train[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_train = df_train[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

rating_1_test =  df_test[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_test = df_test[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

In [13]:
y_train = df_train['target']
rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train = shuffle(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42)

In [14]:
rating_1_train, rating_1_val, \
rating_2_train, rating_2_val, \
fig_1_train, fig_1_val, \
fig_2_train, fig_2_val, \
y_train, y_val = train_test_split(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42, test_size=0.2)

In [15]:
figs_train = np.concatenate([fig_1_train, fig_2_train], axis=1)
figs_val = np.concatenate([fig_1_val, fig_2_val], axis=1)
figs_test = np.concatenate([fig_1_test, fig_2_test], axis=1)

rating_train = np.concatenate([rating_1_train, rating_2_train], axis=1)
rating_val = np.concatenate([rating_1_val, rating_2_val], axis=1)
rating_test = np.concatenate([rating_1_test, rating_2_test], axis=1)





# normalization

In [16]:
mean = rating_train.mean(axis=0)
std = rating_train.std(axis=0)

In [17]:
rating_train = (rating_train-mean)/std
rating_val = (rating_val-mean)/std
rating_test = (rating_test-mean)/std



# Data for torch

In [18]:
class TrainDataset(Dataset):
    def __init__(self, figs, rating, y):
        self.figs = torch.tensor(figs, dtype=torch.float)
        self.rating = torch.tensor(rating, dtype=torch.float)
        self.y = torch.tensor(y.tolist(), dtype=torch.float)
    
    def __len__(self):
        return len(self.figs)
    
    def __getitem__(self, idx):
        return {
            'figs': self.figs[idx],
            'rating': self.rating[idx],
            'y': self.y[idx]
        }

class TestDataset(Dataset):
    def __init__(self, figs, rating):
        self.figs = torch.tensor(figs, dtype=torch.float)
        self.rating = torch.tensor(rating, dtype=torch.float)
    
    def __len__(self):
        return len(self.figs)
    
    def __getitem__(self, idx):
        return {
            'figs': self.figs[idx],
            'rating': self.rating[idx]
        }
        

In [19]:
def train_epoch(iterator, optimizer, criterion):
        model.train()
        epoch_loss = 0

        for i, batch in tqdm(enumerate(iterator)):
            targets = batch['y'].to(device)
            figs = batch['figs'].to(device)
            rating = batch['rating'].to(device)

            optimizer.zero_grad()

            logits = model(figs, rating).squeeze()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def val_epoch(iterator,criterion):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                targets = batch['y'].to(device)
                figs = batch['figs'].to(device)
                rating = batch['rating'].to(device)

                logits = model(figs, rating).squeeze()
                loss = criterion(logits, targets)
                epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def predict(iterator):
        model.eval()
        preds = []
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                figs = batch['figs'].to(device)
                rating = batch['rating'].to(device)
                pred = model.predict(figs, rating)
                preds += pred.to('cpu').numpy().tolist()

        return preds

In [20]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1figs = nn.Linear(202, 256)
        self.bn1figs = nn.BatchNorm1d(256)        
        self.fc2figs = nn.Linear(256, 128)
        self.bn2figs = nn.BatchNorm1d(128)
        self.fc3figs = nn.Linear(128, 128)
        self.bn3figs = nn.BatchNorm1d(128)
       
        self.fc1rating = nn.Linear(34, 128)
        self.bn1rating = nn.BatchNorm1d(128)
        self.fc2rating = nn.Linear(128, 64)
        self.bn2rating = nn.BatchNorm1d(64)
        self.fc3rating = nn.Linear(64, 64)
        self.bn3rating = nn.BatchNorm1d(64)
        
        self.do = nn.Dropout(0.5)
        self.fc1res = nn.Linear(128+64, 128)
        self.bn1res = nn.BatchNorm1d(128)
        self.fc2res = nn.Linear(128, 64)
        self.bn2res = nn.BatchNorm1d(64)
        self.fc3res = nn.Linear(64, 32)
        self.bn3res = nn.BatchNorm1d(32)
        self.fc4res = nn.Linear(32, 1)
    
    def predict(self, figs, rating):
        result = self.forward(figs, rating)
        result = F.sigmoid(result)
        return result
        
    def forward(self, figs, rating):
        figs = self.fc1figs(figs)
        figs = self.bn1figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        figs = self.fc2figs(figs)
        figs = self.bn2figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        figs = self.fc3figs(figs)
        figs = self.bn3figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        
        rating = self.fc1rating(rating)
        rating = self.bn1rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        rating = self.fc2rating(rating)
        rating = self.bn2rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        rating = self.fc3rating(rating)
        rating = self.bn3rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        concat_data = torch.cat((figs, rating), dim=-1)
        
        result = self.fc1res(concat_data)
        result = self.bn1res(result)
        result = F.relu(result)
        
        result = self.fc2res(result)
        result = self.bn2res(result)
        result = F.relu(result)
        
        result = self.fc3res(result)
        result = self.bn3res(result)
        result = F.relu(result)
        
        result = self.fc4res(result)
        
        return result

In [21]:
batch_size = 64
train_dataset = TrainDataset(figs_train, rating_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TrainDataset(figs_val, rating_val, y_val)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = TestDataset(figs_test, rating_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [22]:
train_dataset[0]

{'figs': tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 1., 0., 0.]),
 'rating': tensor([-0.8310, -0.9681, -0.7451, -0.8300, -0.9681, -0.7450, -

In [23]:
model = NN()
if device == 'cuda':
    model = model.cuda()

In [24]:
epochs = 60
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001*1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5, last_epoch=-1)

In [25]:
best_valid_loss = float('inf')
for epoch in range(1, epochs + 1):
    train_loss = round(train_epoch(train_dataloader, optimizer, criterion), 7)
    valid_loss = round(val_epoch(valid_dataloader, criterion), 7)
    if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), './data/models/result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_more_more_64_more_steps_normalize.pt')

    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    scheduler.step()

45413it [05:50, 129.59it/s]
11354it [00:16, 678.85it/s]
1it [00:00,  5.22it/s]

Epoch: 1 
 Train Loss 0.5268776  Val loss 0.5001532:


45413it [05:47, 130.81it/s]
11354it [00:16, 683.43it/s]
1it [00:00,  5.41it/s]

Epoch: 2 
 Train Loss 0.4958883  Val loss 0.5008768:


45413it [05:45, 131.50it/s]
11354it [00:16, 688.09it/s]
1it [00:00,  5.46it/s]

...

Epoch: 58 
 Train Loss 0.4430851  Val loss 0.4549213:


45413it [05:44, 131.92it/s]
11354it [00:16, 684.05it/s]
0it [00:00, ?it/s]

Epoch: 59 
 Train Loss 0.4431531  Val loss 0.4544686:


45413it [05:44, 131.96it/s]
11354it [00:16, 683.06it/s]

Epoch: 60 
 Train Loss 0.4431702  Val loss 0.4550751:


In [26]:
model.load_state_dict(torch.load('./data/models/result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_more_more_64_more_steps_normalize.pt'))

<All keys matched successfully>

In [27]:
round(val_epoch(valid_dataloader, criterion), 7)

11354it [00:16, 688.90it/s]


0.4538914

In [28]:
res = predict(test_dataloader)

0it [00:00, ?it/s]/home/mpopov/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
14192it [00:18, 785.64it/s]


In [29]:
res = [item[0] for item in res]

In [30]:
df_test['target'] = res

In [31]:
df_test[['id','target']].to_csv('result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_more_more_64_more_steps_normalize.csv', index=False)

In [32]:
!head  result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_more_more_64_more_steps_normalize.csv

id,target
0,0.5402498841285706
1,0.0246985275298357
2,0.27421900629997253
3,0.07322155684232712
4,0.1372326910495758
5,0.02727675624191761
6,0.48711949586868286
7,0.2022111415863037
8,0.9008543491363525


In [33]:
# val
val_preds = predict(valid_dataloader)
val_preds = [item[0] for item in val_preds]
df_val = pd.DataFrame({'score':val_preds, 'target': y_val})
df_val.to_csv('val_result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_more_more_64_more_steps_normalize.csv', index=False)

11354it [00:15, 744.19it/s]
